In [1]:
import pandas as pd
import os
from sklearn import model_selection
from nltk.corpus import movie_reviews
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import string
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import nltk
from sklearn.ensemble import RandomForestClassifier  
import numpy as np
from nltk import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [2]:
## creating stopwords.
stops = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stops.update(punctuation)

In [3]:
os.listdir()

['.ipynb_checkpoints',
 '0000000000002747_test_twitter_x_test.csv',
 '0000000000002747_training_twitter_x_y_train.csv',
 'TSA.ipynb',
 'TSAPredictions',
 'TSAPredictions.csv']

In [4]:
train = pd.read_csv('0000000000002747_training_twitter_x_y_train.csv')
test = pd.read_csv('0000000000002747_test_twitter_x_test.csv')

In [5]:
train


,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [6]:
test

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...
3655,570304244001193984,US Airways,NaN,Anthony_Scerri,NaN,0,@USAirways Been stuck for 40+ minutes due to l...,NaN,2015-02-24 11:28:22 -0800,"Astoria, NY",Quito
3656,567847737061941249,US Airways,NaN,mttdprkr,NaN,0,@USAirways 4 hours... 4 hours... FOUR HOURS. ...,NaN,2015-02-17 16:47:05 -0800,"Vancouver, WA",Pacific Time (US & Canada)
3657,567823564167192576,Virgin America,NaN,miaerolinea,NaN,1,Nice RT @VirginAmerica: The man of steel might...,NaN,2015-02-17 15:11:02 -0800,Worldwide,Caracas
3658,570273819287531520,American,NaN,GoldensPleasure,NaN,0,@AmericanAir Aww Thanks AA..DFW was on GMA up ...,NaN,2015-02-24 09:27:28 -0800,East Coast CT.,Central Time (US & Canada)


In [7]:
train.isna().sum()

tweet_id                      0
airline_sentiment             0
airline                       0
airline_sentiment_gold    10949
name                          0
negativereason_gold       10956
retweet_count                 0
text                          0
tweet_coord               10204
tweet_created                 0
tweet_location             3550
user_timezone              3577
dtype: int64

**Looking at the train and test data, majority of the column is of no use.  
So we can use the column required for data cleaning.**
# Cleaning For Training Data

In [8]:
Y = [i for i in train['airline_sentiment']]
Y

['negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive'

In [9]:
train['text'][0]

'@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled'

In [10]:
X = []
for i in train['text']:
    i = str(i)
    j = word_tokenize(i)
    X.append(j)
X

[['@',
  'SouthwestAir',
  'I',
  'am',
  'scheduled',
  'for',
  'the',
  'morning',
  ',',
  '2',
  'days',
  'after',
  'the',
  'fact',
  ',',
  'yes',
  '..',
  'not',
  'sure',
  'why',
  'my',
  'evening',
  'flight',
  'was',
  'the',
  'only',
  'one',
  'Cancelled',
  'Flightled'],
 ['@',
  'SouthwestAir',
  'seeing',
  'your',
  'workers',
  'time',
  'in',
  'and',
  'time',
  'out',
  'going',
  'above',
  'and',
  'beyond',
  'is',
  'why',
  'I',
  'love',
  'flying',
  'with',
  'you',
  'guys',
  '.',
  'Thank',
  'you',
  '!'],
 ['@',
  'united',
  'Flew',
  'ORD',
  'to',
  'Miami',
  'and',
  'back',
  'and',
  'had',
  'great',
  'crew',
  ',',
  'service',
  'on',
  'both',
  'legs',
  '.',
  'THANKS'],
 ['@', 'SouthwestAir', '@', 'dultch97', 'that', "'s", 'horse', 'radish', '😤🐴'],
 ['@',
  'united',
  'so',
  'our',
  'flight',
  'into',
  'ORD',
  'was',
  'delayed',
  'because',
  'of',
  'Air',
  'Force',
  'One',
  ',',
  'but',
  'the',
  'last',
  'flight',

In [11]:
# from nltk.tokenize import WordPunctTokenizer
# tk = WordPunctTokenizer()

# l = train['text'][0]
# k = tk.tokenize(l)
# k

**Now we do data cleaning.**

In [12]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [13]:
lemmatizer = WordNetLemmatizer()
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops and len(w) > 2 and w.isalpha() and 'https' not in w and 'www.' not in w or '@' not in w and len(w) > 1 and w[0] not in punctuation:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [14]:
train_tweet = [clean_review(tweet) for tweet in X]



In [15]:
# j = 0
# for i in X:
#     print(i)
#     print(type(i))
#     k = word_tokenize(i)
#     print(k)
#     j += 1
#     if j == 1:
#         break



In [16]:
train_tweet[0]

['southwestair',
 'be',
 'schedule',
 'for',
 'the',
 'morning',
 'day',
 'after',
 'the',
 'fact',
 'yes',
 'not',
 'sure',
 'why',
 'my',
 'even',
 'flight',
 'be',
 'the',
 'only',
 'one',
 'cancelled',
 'flightled']

In [17]:
trained_tweet = [' '.join(i) for i in train_tweet]
train_sentiment =[i for i in Y]

In [18]:
trained_tweet


['southwestair be schedule for the morning day after the fact yes not sure why my even flight be the only one cancelled flightled',
 'southwestair see your worker time in and time out go above and beyond be why love fly with you guy thank you',
 'united flew ord to miami and back and have great crew service on both leg thanks',
 'southwestair dultch97 that horse radish 😤🐴',
 'united so our flight into ord be delayed because of air force one but the last flight to sbn be at 8:20 min from now we just land',
 'united why do you load u in this fly sardine can if you knew the pilot be hour late flight incompetent beyond belief',
 'jetblue that be stock response delays not a frustrate a poor cust serv amp be told by ppl to wait amp they come back but do not',
 'jetblue that be nice hoping to rack up enough mile to take trip to seattle and enjoy perfect latte in the city of coffee',
 'united frankly bad customer service ever problems will happen how you deal defines company never again united

# Cleaning For Test Data

In [19]:
X = []
for i in test['text']:
    i = str(i)
    j = word_tokenize(i)
    X.append(j)


In [20]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [21]:
lemmatizer = WordNetLemmatizer()
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops and len(w) > 2 and w.isalpha() and 'https' not in w and 'www.' not in w or '@' not in w and len(w) > 1 and w[0] not in punctuation:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [22]:
test_tweet = [clean_review(tweet) for tweet in X]



In [23]:
test_tweet = [' '.join(i) for i in test_tweet]


In [24]:
test_tweet[0]
len(test_tweet)
len(test)

3660

# Using Countvectorizer

In [25]:
cv = CountVectorizer(max_features = 3000)  ## good result qith 2000
x_train, x_test, y_train, y_test = train_test_split(trained_tweet, train_sentiment)

In [26]:
train1 = cv.fit_transform(x_train)


In [27]:
test1 = cv.transform(x_test)

## Testing with Classifier

In [28]:
svc = SVC()

In [29]:
svc.fit(train1, y_train)



SVC()

In [30]:
ypred = svc.predict(test1)

In [31]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, ypred)

0.7868852459016393

In [32]:
svc.score(test1, y_test)
## this one is best now.
## we will predict the model from here

0.7868852459016393

**OG PREDICTION**

In [33]:
cv1 = CountVectorizer(max_features = 2000)

train_og = cv1.fit_transform(trained_tweet)

In [34]:
test_og = cv1.transform(test_tweet)

In [35]:
svc = SVC()

In [36]:
svc.fit(train_og, train_sentiment)


SVC()

In [37]:
ypred = svc.predict(test_og)

In [38]:
np.savetxt('TSAPredictions1.csv', ypred, fmt = "%s")

##### Using NaiveBayes

In [39]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [40]:
clf.fit(train1, y_train)


MultinomialNB()

In [41]:
y_pred3 = clf.predict(test1)

In [42]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred3)

0.7832422586520947

# Using tfidfCountVectorizer

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), max_features = 3000) ## best core with max_f = 3000 and ngram=(1,2)

In [44]:

train1= tf.fit_transform(x_train)

In [45]:
test1 = tf.transform(x_test)

In [46]:
svc.fit(train1, y_train)

SVC()

In [47]:
y1 = svc.predict(test1)

In [48]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y1)

0.7876138433515483

# Now we use the nltk 

In [49]:
XTRAIN = []
for i in range(len(x_train)):
    k = word_tokenize(x_train[i])
    k = list(k)
    XTRAIN.append((k, y_train[i]))
XTRAIN

[(['usairways',
   'americanair',
   'according',
   'to',
   'your',
   'costumer',
   'support',
   'your',
   'website',
   'be',
   'hold',
   'my',
   'money',
   'hostage',
   'and',
   'you',
   'cant',
   'do',
   'anything',
   'about',
   'it'],
  'negative'),
 (['americanair', 'thank', 'you', 'you', 'will', 'see', 'me'], 'positive'),
 (['jetblue',
   'lot',
   'of',
   'people',
   'scratch',
   'head',
   'and',
   'look',
   'at',
   'screen',
   'ignore',
   'passenger'],
  'negative'),
 (['virginamerica',
   'cool',
   'picture',
   'of',
   'another',
   'virginamerica',
   'plane',
   'off',
   'our',
   'wing',
   'what',
   'site',
   'http'],
  'positive'),
 (['americanair',
   'be',
   'profit',
   'in',
   'the',
   'billion',
   'and',
   'throw',
   'crap',
   'at',
   'the',
   'people',
   'who',
   'put',
   'them',
   'there',
   'american',
   'be',
   'hurt',
   'brand'],
  'negative'),
 (['united',
   'book',
   'flight',
   'for',
   'for',
   '779',
   

In [50]:
XTEST = []
for i in range(len(x_test)):
    k = word_tokenize(x_test[i])
    k = list(k)

    XTEST.append((k, y_test[i]))
XTEST

[(['usairways',
   'still',
   'in',
   'yuma',
   'wait',
   'for',
   '2702',
   'to',
   'go',
   'looks',
   'like',
   'might',
   'miss',
   '2696',
   'to',
   'montereyregionalairport',
   'need',
   'option',
   'please'],
  'negative'),
 (['americanair',
   'the',
   'status',
   'on',
   'my',
   'itinerary',
   'say',
   'on',
   'request',
   'in',
   'red',
   'what',
   'that',
   'mean'],
  'neutral'),
 (['americanair',
   'dogbuckeye',
   'no',
   'be',
   'on',
   'hold',
   'for',
   '1/2',
   'hour',
   'diff',
   'agent',
   'each',
   'w/',
   'different',
   'information',
   'frustrate',
   'kept',
   'get',
   'txfd'],
  'negative'),
 (['united',
   'probably',
   'would',
   "n't",
   'be',
   'do',
   'it',
   'have',
   'someone',
   'return',
   'my',
   'call',
   'or',
   'email',
   'case',
   'number',
   '8273993'],
  'negative'),
 (['southwestair',
   'flight',
   '4040',
   'atl',
   'to',
   'lga',
   'cancelled',
   'flightled',
   'any',
   'sugge

In [51]:
all_words = []
for d in XTRAIN: 
    all_words +=d[0]
    

In [52]:
all_words

['usairways',
 'americanair',
 'according',
 'to',
 'your',
 'costumer',
 'support',
 'your',
 'website',
 'be',
 'hold',
 'my',
 'money',
 'hostage',
 'and',
 'you',
 'cant',
 'do',
 'anything',
 'about',
 'it',
 'americanair',
 'thank',
 'you',
 'you',
 'will',
 'see',
 'me',
 'jetblue',
 'lot',
 'of',
 'people',
 'scratch',
 'head',
 'and',
 'look',
 'at',
 'screen',
 'ignore',
 'passenger',
 'virginamerica',
 'cool',
 'picture',
 'of',
 'another',
 'virginamerica',
 'plane',
 'off',
 'our',
 'wing',
 'what',
 'site',
 'http',
 'americanair',
 'be',
 'profit',
 'in',
 'the',
 'billion',
 'and',
 'throw',
 'crap',
 'at',
 'the',
 'people',
 'who',
 'put',
 'them',
 'there',
 'american',
 'be',
 'hurt',
 'brand',
 'united',
 'book',
 'flight',
 'for',
 'for',
 '779',
 'was',
 'charge',
 '389.50',
 'twice',
 'means',
 'ca',
 "n't",
 'redeem',
 'cc',
 'point',
 'properly',
 'would',
 'have',
 'go',
 'elsewhere',
 'americanair',
 'you',
 'call',
 'back',
 'just',
 'to',
 'put',
 'me',
 '

In [53]:
freq = nltk.FreqDist(all_words)

In [54]:
common = freq.most_common(2000)

In [55]:
features = [i[0] for i in common]

In [56]:
def get_feature_dict(words):
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set
    return current_features

In [57]:
training_data = [(get_feature_dict(i), category) for i, category in XTRAIN]

In [58]:
testing_data = [(get_feature_dict(i), category) for i, category in XTEST]

**passing nltk classifier**

In [59]:
from nltk import NaiveBayesClassifier
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.ensemble import RandomForestClassifier  


In [60]:
svc = SVC()

In [61]:
classifier_sklearn = SklearnClassifier(svc)

In [62]:
classifier_sklearn.train(training_data)

<SklearnClassifier(SVC())>

In [63]:
nltk.classify.accuracy(classifier_sklearn, testing_data)

0.7934426229508197

**Using RandomForest**

In [64]:
rf = RandomForestClassifier()

In [65]:
classifier_sklearn1 = SklearnClassifier(rf)

In [66]:
classifier_sklearn1.train(training_data)

<SklearnClassifier(RandomForestClassifier())>

In [67]:
nltk.classify.accuracy(classifier_sklearn1, testing_data)

0.7544626593806921

**Using Naivebayes**

In [68]:
classifier = NaiveBayesClassifier.train(training_data)

In [69]:
nltk.classify.accuracy(classifier, testing_data)

0.7836065573770492

**tadmMaxent Classifier**